<a href="https://colab.research.google.com/github/mandie021/big-data-challenge/blob/main/level-1/level_1_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
#import postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-06 19:30:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.75MB/s    in 0.2s    

2022-03-06 19:30:07 (5.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
#create session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
game_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
game_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
#Remove duplicates and na values
game_df = game_df.dropna()
game_df = game_df.dropDuplicates()

In [6]:
game_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Count the number of records (rows) in the dataset
game_df.count()

1785886

In [8]:
# Transform the dataset to fit the tables in the schema file
#imports and review id table
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date

review_id_table = game_df.select(["review_id",  "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1004PYTPK6ELD|   38281029|B00004WHWF|      89143877| 2002-01-23|
|R100EZDMO39LBZ|    1386752|B00OZBFUBY|     872540442| 2015-06-10|
|R1011I65X7RSKT|   41907229|B00006ISBU|     654790631| 2003-12-23|
|R101V84BKDOR1I|   12034223|B001KX5042|     279727821| 2014-08-08|
|R101VJUP2TFB3Y|   31532612|B009DL2TBA|     586138868| 2015-01-05|
|R1026A0A5F4D42|   30645349|B00FEKQZE6|     681125586| 2014-04-21|
|R102X78AGOZJY3|   20191662|B000ZK9QCS|     699333646| 2010-03-26|
|R104THW8CUUZXC|   20910843|B00DHF39EO|     100570084| 2015-06-15|
|R105YSA8EGSJ8D|    2435449|B0053B5RGI|     413365293| 2015-02-26|
|R106QTPT8YYZ1P|    4159081|B000TGB4UA|      17513844| 2015-04-22|
|R106UDOJ2QSWOR|   11343413|B00KVP78FE|     177244653| 2014-11-14|
|R109G94NZM2I7R|   22043357|B004XACA60|     432872195| 2012-03

In [9]:
#make columns int
review_id_table = review_id_table.withColumn("customer_id", review_id_table["customer_id"].cast(IntegerType()))
review_id_table = review_id_table.withColumn("product_parent", review_id_table["product_parent"].cast(IntegerType()))

In [10]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [11]:
# Transform the dataset to fit the tables in the schema file
#imports and products table
from pyspark import SparkFiles
products = game_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [12]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [13]:
# Transform the dataset to fit the tables in the schema file
#imports and customers table
customers = game_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    5401088|             5|
|   37964777|            16|
|   20872523|             1|
|   44777937|             4|
|    2384511|             1|
|   41500251|             1|
|   34029693|             1|
|   40005173|             2|
|   12125016|             2|
|   21226328|             3|
|    6004768|             2|
|   14445274|             4|
|   41665035|             1|
|   16995171|             1|
|   25044130|             1|
|   11073960|             2|
|   38642776|             8|
|    6199838|             6|
|   19207466|             3|
|    1760318|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
#convert items to int
customers = customers.withColumn("customer_id", customers["customer_id"].cast(IntegerType()))
customers = customers.withColumn("customer_count", customers["customer_count"].cast(IntegerType()))

In [15]:
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [16]:
# Transform the dataset to fit the tables in the schema file
#imports and vine table
vine_table = game_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1004PYTPK6ELD|          5|            0|          0|   N|
|R100EZDMO39LBZ|          4|            0|          0|   N|
|R1011I65X7RSKT|          5|            2|          2|   N|
|R101V84BKDOR1I|          5|            0|          0|   N|
|R101VJUP2TFB3Y|          5|            0|          0|   N|
|R1026A0A5F4D42|          5|            5|          6|   N|
|R102X78AGOZJY3|          5|            0|          0|   N|
|R104THW8CUUZXC|          1|            2|          6|   N|
|R105YSA8EGSJ8D|          4|            0|          0|   N|
|R106QTPT8YYZ1P|          5|            1|          1|   N|
|R106UDOJ2QSWOR|          5|            0|          0|   N|
|R109G94NZM2I7R|          5|            0|          1|   N|
|R10A0VLQRM6JIA|          3|            3|          3|   N|
|R10ADC7LCXO3V2|          1|            

In [17]:
#convert items to int
vine_table = vine_table.withColumn("star_rating", vine_table["star_rating"].cast(IntegerType()))
vine_table = vine_table.withColumn("helpful_votes", vine_table["helpful_votes"].cast(IntegerType()))
vine_table = vine_table.withColumn("total_votes", vine_table["total_votes"].cast(IntegerType()))

In [18]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [21]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bigdatahwdb.crkggtvwjikn.us-east-1.rds.amazonaws.com:5432/hw_lvl1_db"
config = {"user":"root", 
          "password": "****", 
          "driver":"org.postgresql.Driver"}


In [22]:
# Write DataFrame to review_id table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [23]:
# Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [24]:
# Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)